In [2]:
import pandas as pd
import numpy as np
import os

In [3]:
df = pd.read_csv("./data_rating.csv", encoding='cp949')

In [4]:
dataset=df.dropna(axis=0)

In [5]:
dataset

,User ID2,User ID3,Location Frequency,Activity,Main Activity Time,Cluster,User ID,Venue ID,Venue category ID,Venue category name,Latitude,Longitude,Timezone offset in minutes,UTC time,Rating
0,0,1,0,0,22,1,2454,4abc1f51f964a520798620e3,4bf58dd8d48988d1ce941735,Seafood Restaurant,40.781558,-73.975792,-240,Wed Apr 04 23:31:31 +0000 2012,0.0
1,1,1,0,0,22,1,3660,4d4ac10da0ef54814b6ffff6,4bf58dd8d48988d157941735,American Restaurant,40.784018,-73.974524,-240,Sat Apr 07 17:42:24 +0000 2012,0.0
2,2,1,0,0,22,1,5603,4db44994cda1c57c82583709,4bf58dd8d48988d1f1931735,General Entertainment,40.739398,-73.993210,-240,Sun Apr 08 18:20:29 +0000 2012,0.0
3,3,1,0,0,22,1,5783,4a541923f964a52008b31fe3,4bf58dd8d48988d14e941735,American Restaurant,40.785677,-73.976498,-240,Sun Apr 08 20:02:10 +0000 2012,0.0
4,4,1,0,0,22,1,6696,40f1d480f964a5205b0a1fe3,4bf58dd8d48988d143941735,Breakfast Spot,40.719929,-74.008532,-240,Mon Apr 09 16:20:52 +0000 2012,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3004,10601,185,290,165,21,1,64875,4d8f74cf1716a143e3ed4af7,4bf58dd8d48988d163941735,Park,40.965417,-74.062920,-240,Sun May 13 17:12:58 +0000 2012,1.0
3005,10602,185,290,165,21,1,64946,4cf526178333224b089d1e8e,4bf58dd8d48988d1e5931735,Music Venue,40.965322,-74.062803,-240,Sun May 13 17:43:59 +0000 2012,1.0
3006,10603,185,290,165,21,1,65035,4ef6571c722e34060e72b97f,4bf58dd8d48988d14e941735,American Restaurant,40.965329,-74.062924,-240,Sun May 13 18:22:58 +0000 2012,1.0
3007,10604,185,290,165,21,1,65183,4bd7282e5631c9b6e315a730,4bf58dd8d48988d124941735,Office,40.964622,-74.067857,-240,Sun May 13 19:33:08 +0000 2012,1.0


In [6]:
# 파일 생성
dataset.to_csv('./active_cf.csv')

평점 부여

In [7]:
# import pandas as pd

In [8]:
# 데이터셋 파일 경로
# file_path = 'C:/Users/seohe/OneDrive/바탕 화면/thesis/dataset_cf.csv'

In [9]:
# 데이터셋 로드
# data = pd.read_csv(file_path)

In [10]:
# # 각 장소의 방문 횟수를 계산하여 인기도 기반 평점 부여
# venue_count = data['Venue ID'].value_counts()
# venue_count_dict = venue_count.to_dict()
# data['rating'] = data['Venue ID'].apply(lambda x: venue_count_dict[x])
# data.to_csv('C:/Users/seohe/OneDrive/바탕 화면/thesis/dataset_cf.csv', index=True)

In [11]:
# Surprise 데이터셋으로 변환
from surprise import Dataset
from surprise import Reader

In [12]:
reader = Reader(line_format='user item rating timestamp', sep=',', rating_scale=(1, 5))
data = Dataset.load_from_df(dataset[['User ID', 'Venue ID', 'Rating']], reader=reader)

샘플링 -> 협업필터링 구현

In [13]:
from surprise import SVD
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import train_test_split
from surprise import accuracy
import pandas as pd

In [14]:
# 데이터 로드
df = pd.read_csv('./active_cf.csv')

In [15]:
# 데이터 형식 지정
reader = Reader(rating_scale=(1, 5))

In [16]:
# 데이터프레임을 데이터셋으로 변환
data = Dataset.load_from_df(df[['User ID', 'Venue ID', 'Rating']], reader)

In [17]:
# 학습용 데이터와 테스트용 데이터로 분할
trainset, testset = train_test_split(data, test_size=0.2, random_state=42)

In [18]:
# SVD 모델 학습
model = SVD(n_factors=100, n_epochs=20, random_state=42)
model.fit(trainset)

In [19]:
# 테스트 데이터로 모델 평가
predictions = model.test(testset)
print('RMSE:', accuracy.rmse(predictions))

RMSE: 0.5692
RMSE: 0.5692460493066992


In [20]:
from surprise.model_selection import train_test_split
from surprise import Dataset

# 데이터셋 불러오기
data = Dataset.load_builtin('ml-100k')

# 랜덤 샘플링을 통해 학습셋과 검증셋 나누기
trainset, testset = train_test_split(data, test_size=0.2, random_state=42)

# 모델 학습 및 예측
algo = SVD()
algo.fit(trainset)
predictions = algo.test(testset)

# 평가
accuracy.rmse(predictions)

RMSE: 0.9347


0.9346820465297705

정밀도, 재현율, f

In [21]:
from surprise import accuracy
from sklearn.metrics import precision_recall_fscore_support

# 학습 데이터로 모델 학습
model.fit(trainset)

# 테스트 데이터로 모델 평가
predictions = model.test(testset)

# 예측값과 실제값을 리스트로 저장
pred = []
true = []
for uid, iid, r_ui, est, _ in predictions:
    pred.append(est)
    true.append(r_ui)

# NaN 값 제거
pred = np.nan_to_num(pred)

# 정밀도, 재현율, F1-score 계산
precision, recall, f1_score, _ = precision_recall_fscore_support(true, np.around(pred), average='weighted')
print("Precision: ", precision)
print("Recall: ", recall)
print("F1-score: ", f1_score)

Precision:  0.46750595873690015
Recall:  0.41815
F1-score:  0.3782788718945031
